In [2]:
import asyncio
import socketio
import random
import json
sio = socketio.AsyncClient()

map = []
snakes = {}
foods = []
player_id = None
FRAME_RATE = 0.1


def direction_to_string(direction):
    if direction == [0, -1]:
        return "up"
    if direction == [0, 1]:
        return "down"
    if direction == [-1, 0]:
        return "left"
    if direction == [1, 0]:
        return "right"
    return None


@sio.event
async def connect():
    print("Connected to the server")
    await sio.emit("register", {"player_id": player_id})


@sio.event
async def disconnect():
    print("Disconnected from the server")


@sio.event
async def state(data):
    global map, snakes, foods, player_id
    data = json.loads(data)
    map = data["map"]
    snakes = data["snakes_pos"]
    foods = data["foods_pos"]
    
    if player_id is None: return
    
    if str(player_id) not in snakes.keys():
        
        await sio.emit("exit")
        await sio.disconnect()
        return
    player_pos = snakes[str(player_id)]["body"][0]
    player_direction = direction_to_string(snakes[str(player_id)]["direction"])
    print(player_pos, player_direction)


@sio.event
async def register(data):
    print("Registered as player", data["player_id"])
    global player_id
    player_id = data["player_id"]
    print(f"Registered as player {player_id}")
    # After registration, start sending moves
    asyncio.create_task(send_moves())

async def send_moves():
    
    while True:
        global map, snakes, player_id, foods
        # Find the player's current position
        player_pos = None
        player_direction = None
        if str(player_id) in snakes.keys():
            player_pos = snakes[str(player_id)]["body"][0]
            player_direction = direction_to_string(snakes[str(player_id)]["direction"])
        # If the player's position was not found, don't send a move message
        if player_pos is None or player_direction is None:
            await asyncio.sleep(FRAME_RATE)
            continue

        # print (player_pos, player_direction)
        # Determine possible moves
        directions = []
        if (
            player_pos[0] > 0
            and (
                map[player_pos[0] - 1][player_pos[1]] == ""
                or map[player_pos[0] - 1][player_pos[1]] == "food"
            )
            and player_direction != "right"
        ):
            directions.append("left")
        if (
            player_pos[0] < len(map) - 1
            and (
                map[player_pos[0] + 1][player_pos[1]] == ""
                or map[player_pos[0] + 1][player_pos[1]] == "food"
            )
            and player_direction != "left"
        ):
            directions.append("right")
        if (
            player_pos[1] > 0
            and (
                map[player_pos[0]][player_pos[1] - 1] == ""
                or map[player_pos[0]][player_pos[1] - 1] == "food"
            )
            and player_direction != "down"
        ):
            directions.append("up")
        if (
            player_pos[1] < len(map[0]) - 1
            and (
                map[player_pos[0]][player_pos[1] + 1] == ""
                or map[player_pos[0]][player_pos[1] + 1] == "food"
            )
            and player_direction != "up"
        ):
            directions.append("down")

        # If there are no possible moves, don't send a move message
        if not directions:
            await asyncio.sleep(FRAME_RATE)
            continue

        # Choose a random direction from the possible moves
        random_direction = random.choice(directions)
        message = {"type": "move", "direction": random_direction}
        await sio.emit("move", message)
        await asyncio.sleep(FRAME_RATE)  # pause for a second between messages


async def main():
    global player_id
    player_id = input("Please enter your player ID: ")
    await sio.connect("http://140.112.106.45:3000/")
    # await sio.connect("http://localhost:3000")
    await sio.wait()


import nest_asyncio

nest_asyncio.apply()

await main()

Connected to the server
Registered as player yym
Registered as player yym
[15, 9] right
[15, 8] up
[14, 8] left
[14, 9] down
[15, 9] right
[15, 10] down
[15, 11] down
[14, 11] left
[14, 10] up
[13, 10] left
[13, 9] up
[13, 8] up
[13, 7] up
[13, 6] up
[14, 6] right
[15, 6] right
[16, 6] right
[16, 7] down
[16, 8] down
[16, 9] down
[15, 9] left
[15, 10] down
[15, 11] down
[16, 11] right
[17, 11] right
[17, 12] down
[16, 12] left
[16, 13] down
[17, 13] right
[18, 13] right
[18, 12] up
[17, 12] left
[16, 12] left
[15, 12] left
[15, 13] down
[14, 13] left
[13, 13] left
[13, 14] down
[14, 14] right
[14, 15] down
[13, 15] left
[12, 15] left
[12, 16] down
[12, 17] down
[12, 18] down
[11, 18] left
[11, 17] up
[12, 17] right
[13, 17] right
[13, 16] up
[13, 15] up
[13, 14] up
[13, 13] up
[14, 13] right
[14, 12] up
[14, 11] up
[13, 11] left
[12, 11] left
[11, 11] left
[10, 11] left
[10, 12] down
[10, 13] down
[11, 13] right
[11, 14] down
[12, 14] right
[13, 14] right
[13, 13] up
[12, 13] left
[12,